In [22]:
import pickle
import numpy as np
import torch
from torchvision import datasets, transforms
import torch.nn as nn
from importlib import reload
import MobileNet_v3
import functions
import datafunc
import config
config = reload(config)
MobileNet_v3 = reload(MobileNet_v3)
functions = reload(functions)
datafunc = reload(datafunc)
from functions import train, accuracy
from datafunc import MyDataLoader, train_test_split
from MobileNet_v3 import get_model as gm3

device = torch.device(config.DEVICE)

torch.cuda.empty_cache()

In [23]:
mobilenet = gm3(100, 'small', 0.75, 0.8, 3).to(device)

In [24]:
# dataset settings
batch_size = 32
IM_SIZE = 48  # resize image
NORMALIZE = ([0.24, 0.25, 0.26],
             [.31, .32, .33])


train_transformer = transforms.Compose([
    # transforms.Resize(IM_SIZE),
    transforms.Pad((224 - 48)//2),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(*NORMALIZE)
])


test_transformer = transforms.Compose([
    transforms.Pad((224 - 48)//2),
    # transforms.Resize(IM_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(*NORMALIZE)
])

cifar_train = datasets.CIFAR100('data/',
                             transform=train_transformer,
                             download=True)

with open('classes_name.pkl', 'wb') as f:
    pickle.dump(cifar_train.classes, f)

cifar_val = datasets.CIFAR100('data/',
                               transform=test_transformer,
                               train=True)

train_indices, val_indices = \
    train_test_split(np.arange(len(cifar_train)), .75, cifar_train.targets)

train_loader = MyDataLoader(cifar_train, batch_size, train_indices, shuffle=True)
val_loader = MyDataLoader(cifar_val, batch_size, val_indices, shuffle=True)

Files already downloaded and verified


In [25]:
LR = 1e-2
OPTIM_MOMENTUM = 0.9
WEIGHT_DECAY = 1e-5  # l2 weight decay

optimizer = torch.optim.RMSprop(mobilenet.parameters(),
                                lr=LR,
                                momentum=OPTIM_MOMENTUM,
                                weight_decay = WEIGHT_DECAY
                            )
loss_func = nn.NLLLoss()


step_size = 3
gamma = 0.1

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma=gamma,
                                            last_epoch=-1)

In [26]:
EPOCHS= 10
train_history, best_parameters = \
    train(mobilenet, train_loader, loss_func, optimizer, EPOCHS, accuracy, val_loader)

Mode: T |Loss:   30.67 |Metric:  0.01024 |: 100%|██████████| 1172/1172 [06:08<00:00,  3.18it/s]
Mode: V |Loss:   5.486 |Metric:     0.0 |:   0%|          | 1/391 [00:01<07:14,  1.11s/it]


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 4.00 GiB total capacity; 2.30 GiB already allocated; 5.77 MiB free; 2.35 GiB reserved in total by PyTorch)

In [ ]:
torch.save({'model_state_dict': best_parameters}, 'model.torch')

